<a href="https://colab.research.google.com/github/sualeh/introduction-to-chatgpt-api/blob/main/local-vector-database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------

> **How to Run This Notebook**

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. If you are running the notebook locally in Visual Studio Code or other IDE, create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Otherwise, if you are running in Google Colab, create a secret called `OPENAI_API_KEY` and set it to the value of your OpenAI API key.

Run the code below to read the key.


In [ ]:
%pip install -qq python-dotenv

from os import environ as env
from dotenv import load_dotenv
import logging

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
load_dotenv()
try:
  # Attempt to read OPENAI_API_KEY from a Google Colab secret
  from google.colab import userdata
  env['OPENAI_API_KEY'] = env.get('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
except ModuleNotFoundError:
  print("Not running in Google Colab")
  # No action - rely on the OPENAI_API_KEY environmental variable

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)



----------

# Vector Databases

## Define File Loading Functions

Define functions to load and process PDF and text files, and test them

In [ ]:
%pip install -qq langchain langchain-community

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env.params")

DOCS_DIRECTORY = os.getenv("DOCS_DIRECTORY")

In [ ]:
import glob
import os

def get_files_from_directory(directory_path: str, extensions=['.pdf', '.txt']) -> list[str]:
    """
    Get all files with specified extensions from a directory.
    
    Args:
        directory_path (str): Path to the directory containing files.
        extensions (list): List of file extensions to include.
    
    Returns:
        list: List of file paths.
    """
    all_files = []
    for ext in extensions:
        files = glob.glob(os.path.join(directory_path, f"*{ext}"))
        all_files.extend(files)
    return all_files

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader

def load_document(file_path: str) -> list:
    """
    Load a document based on its file extension.
    
    Args:
        file_path (str): Path to the file.
    
    Returns:
        list: List of document objects.
    """
    _, file_extension = os.path.splitext(file_path)
    
    if file_extension.lower() == '.pdf':
        loader = PyPDFLoader(file_path)
        return loader.load()
    
    elif file_extension.lower() == '.txt':
        loader = TextLoader(file_path)
        return loader.load()
    
    else:
        logger.error(f"Unsupported file format: {file_extension}")
        return []

In [ ]:
def load_documents_from_directory(directory_path):
    """
    Load all documents from a directory.
    
    Args:
        directory_path (str): Path to the directory.
    
    Returns:
        list: List of documents.
    """
    files = get_files_from_directory(directory_path)
    
    all_documents = []
    for file_path in files:
        logger.info(f"Loading {file_path}...")
        documents = load_document(file_path)
        all_documents.extend(documents)
    
    logger.info(f"Loaded {len(all_documents)} document chunks.")
    return all_documents

In [ ]:
def print_document_chunks(documents, limit=3):
    print()
    for index, chunk in enumerate(documents):
        if index > limit:
            break
        print(f"------ CHUNK {index+1} -------------------------------------------------")
        print(chunk.metadata)
        print()
        print(chunk.page_content[:100])
        print("... (skipping content) ...")
        print(chunk.page_content[-100:])
        print()

Run the code for loading files

In [ ]:
documents = load_documents_from_directory(DOCS_DIRECTORY)

print_document_chunks(documents)


## Text Splitting

Next, we'll split the documents into smaller chunks for better embedding and retrieval.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, chunk_overlap=100):
    """
    Split documents into smaller chunks.
    
    Args:
        documents (list): List of documents.
        chunk_size (int): Size of each chunk.
        chunk_overlap (int): Overlap between chunks.
    
    Returns:
        list: List of document chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    chunks = text_splitter.split_documents(documents)
    logger.info(f"Split into {len(chunks)} chunks.")
    return chunks

Look at the chunks of text.

In [ ]:
chunks = split_documents(documents)

print_document_chunks(chunks)

## Create a Vector Database

Now, let's create functions to build and save our vector database.

In [ ]:
%pip install -qq faiss-cpu faiss-gpu langchain-openai

In [ ]:
from langchain.vectorstores import FAISS

def create_vector_db(chunks, embeddings_model, save_path=None):
    """
    Create a vector database from document chunks.
    
    Args:
        chunks (list): List of document chunks.
        embeddings_model: Model to create embeddings.
        save_path (str, optional): Path to save the vector database.
    
    Returns:
        FAISS: FAISS vector database.
    """
    vector_db = FAISS.from_documents(chunks, embeddings_model)
    
    if save_path:
        vector_db.save_local(save_path)
        logger.info(f"Vector database saved to {save_path}")
    
    return vector_db

Create vector database

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env.params")

VECTOR_DB_PATH = os.getenv("VECTOR_DB_PATH")

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

vector_db = create_vector_db(chunks, embeddings_model, save_path=VECTOR_DB_PATH)
print(vector_db)

## Query Function

Let's create a function to query our vector database.

In [ ]:
def query_vector_db(query, vector_db, k=5):
    """
    Query the vector database for similar documents.
    
    Args:
        query (str): Query string.
        vector_db: Vector database.
        k (int): Number of results to return.
    
    Returns:
        list: List of documents and their similarities.
    """
    results = vector_db.similarity_search_with_score(query, k=k)
    
    return results

Query the vector database to get documents and their similarities.

In [ ]:
query = "What are the main topics covered in 'Who Moved the Cheese'?"

results = query_vector_db(query, vector_db, k=3)

print_document_chunks([results for results, _ in results])